# Create a Local Docker Image
In this section, we will create an IoT Edge module, a Docker container image with an HTTP web server that has a scoring REST endpoint.

## Get Global Variables

In [1]:
import sys
sys.path.append('../common')
from env_variables import *

## Create Web Application & Inference Server for Our ML Solution

1. To change the inference model that will be used in this sample, change the variable `IS_MODEL_NAME` based on your preferred model. Note that the model must be downloaded, as instructed in the [previous section](create_openvino_inference_engine.ipynb). The default model is         "person-vehicle-bike-detection-crossroad-1016".

2. The variable assignment of `IS_TARGET_DEVICE` indicates what type of hardware acceleration you would like to use on your IoT Edge device. You can choose from the following choices:
    * "CPU" for Intel® CPU acceleration  
    * "MYRIAD" for Intel® VPU acceleration
    * "GPU" for Intel® GPU acceleration
    * "FPGA" for Intel® FPGA acceleration

    Change the variable `IS_TARGET_DEVICE` as needed; the default is "CPU". 

3. The variable assignment of `IS_MODEL_PRECISION` indicates what type of model precision you would like to use. The default is "FP32" for this sample. However, please [check the Intel documentation](https://docs.openvinotoolkit.org/latest/_docs_IE_DG_supported_plugins_Supported_Devices.html) for which output precision is supported for your desired hardware acceleration.

4. Insert the subscription key for your Azure Cognitive Services Optical Character Recognition (OCR) instance (see https://docs.microsoft.com/en-us/azure/cognitive-services/computer-vision/concept-recognizing-text) on the `VISION-API-SUBSCRIPTION-KEY` placeholder on the next cell.

In [2]:
%%writefile $isSolutionPath/app.py
import threading
import cv2
from PIL import Image
import numpy as np
import io
import os
import json
import logging
import linecache
from score import MLModel, PrintGetExceptionDetails
from flask import Flask, request, Response, send_file, make_response
import math
import re
import time
import http.client
import urllib.request
import urllib.parse
import urllib.error
import requests
import io
import json
import base64


def license_plate_detection_on_frame(cvImage):
    output_path = "/home/visionadmin/source/cropped_license_plates"
    detectedObjects = inferenceEngine.score(cvImage)
    if len(detectedObjects) > 0:
        confidence = 0.9
        results = []
        height, width, channels = cvImage.shape
        for detected in detectedObjects:
            label = float(detected["entity"]["tag"]["value"])
            if label == 2.0: # 1.0=Car; 2.0=Licence plate
                conf = float(detected["entity"]["tag"]["confidence"])
                if conf > confidence:
                    results.append(detected)
        for result in results:
            bbox = result['entity']['box']
            left = float(bbox['l'])
            top = float(bbox['t'])
            bbox_width = float(bbox['w'])
            bbox_height = float(bbox['h'])
            left = math.ceil(left * width)
            top = math.ceil(top * height)
            bbox_width = math.ceil(float(bbox['w']) * width)
            bbox_height = math.ceil(float(bbox['h']) * height)
            bottom = top + bbox_height
            right = left + bbox_width
            color = (255, 0, 0)  # Blue in BGR
            top_left = (left, top)
            bottom_right = (right, bottom)
            thickness = 2
            cvImage = cv2.rectangle(cvImage, top_left, bottom_right, color, thickness)
            crop_image = cvImage[top:bottom, left:right]
            if crop_image.size > 0:
                file_name = "crop_" + str(top) + "_" + str(left) + ".jpg"
                output = os.path.join(output_path, file_name)
                cv2.imwrite(output, crop_image)
    return cvImage


def openvino_license_plate_recognition_on_frame(cvImage):
    output_path = "/home/visionadmin/source/cropped_license_plates"
    detectedObjects = inferenceEngine.score(cvImage)
    if len(detectedObjects) > 0:
        confidence = 0.9
        results = []
        height, width, channels = cvImage.shape
        for detected in detectedObjects:
            label = float(detected["entity"]["tag"]["value"])
            if label == 2.0: # 1.0=Car; 2.0=Licence plate
                conf = float(detected["entity"]["tag"]["confidence"])
                if conf > confidence:
                    results.append(detected)
        recogEngine = MLModel(modelName=RECOGNITION_MODEL_NAME,
                            modelPrecision=IS_MODEL_PRECISION, 
                            targetDev=IS_TARGET_DEVICE)
        for result in results:
            bbox = result['entity']['box']
            left = float(bbox['l'])
            top = float(bbox['t'])
            bbox_width = float(bbox['w'])
            bbox_height = float(bbox['h'])
            left = math.ceil(left * width)
            top = math.ceil(top * height)
            bbox_width = math.ceil(float(bbox['w']) * width)
            bbox_height = math.ceil(float(bbox['h']) * height)
            bottom = top + bbox_height
            right = left + bbox_width
            color = (255, 0, 0)  # Blue in BGR
            top_left = (left, top)
            bottom_right = (right, bottom)
            thickness = 2
            cvImage = cv2.rectangle(cvImage, top_left, bottom_right, color, thickness)
            crop_image = cvImage[top:bottom, left:right]
            if crop_image.size > 0:
                file_name = "crop_" + str(top) + "_" + str(left) + ".jpg"
                output = os.path.join(output_path, file_name)
                cv2.imwrite(output, crop_image)
                h, w, c = crop_image.shape
                crop_image = np.reshape(crop_image, (c, h, w))
                plates = recogEngine.score(crop_image)
                scored = []
                for plate in plates:
                    conf = float(detected["entity"]["tag"]["confidence"])
                    if conf > 0.2:
                        scored.append(plate)
    detectedObjects = inferenceEngine.score(cvImage)
    return cvImage


def openvino_cognitiveservices_license_plate_recognition_on_frame(cvImage):
    output_path = "/home/visionadmin/source/cropped_license_plates"
    headers = {
        'Content-Type': 'application/octet-stream', #  multipart/form-data
        'Ocp-Apim-Subscription-Key': '<SUBSCRIPTION-KEY>'
    }
    detectedObjects = inferenceEngine.score(cvImage)
    if len(detectedObjects) > 0:
        confidence = 0.3
        results = []
        height, width, channels = cvImage.shape
        for detected in detectedObjects:
            label = float(detected["entity"]["tag"]["value"])
            if label == 2.0: # 1.0=Car; 2.0=Licence plate
                conf = float(detected["entity"]["tag"]["confidence"])
                if conf > confidence:
                    results.append(detected)
        for result in results:
            bbox = result['entity']['box']
            left = float(bbox['l'])
            top = float(bbox['t'])
            bbox_width = float(bbox['w'])
            bbox_height = float(bbox['h'])
            left = math.ceil(left * width)
            top = math.ceil(top * height)
            bbox_width = math.ceil(float(bbox['w']) * width)
            bbox_height = math.ceil(float(bbox['h']) * height)
            bottom = top + bbox_height
            right = left + bbox_width
            color = (255, 0, 0)  # Blue in BGR
            top_left = (left, top)
            bottom_right = (right, bottom)
            thickness = 2
            cvImage = cv2.rectangle(cvImage, top_left, bottom_right, color, thickness)
            crop_image = cvImage[top:bottom, left:right]
            if crop_image.size > 0:
                file_name = "crop_" + str(top) + "_" + str(left) + ".jpg"
                output = os.path.join(output_path, file_name)
                h, w, c = crop_image.shape
                if h < 50:
                    ratio = w / h
                    new_h = 60
                    new_w = math.ceil(new_h * ratio)
                    crop_image = cv2.resize(crop_image, (new_w, new_h), cv2.INTER_AREA)
                cv2.imwrite(output, crop_image)
                try:
                    encoded_image = cv2.imencode(".jpg", crop_image)[1].tobytes()
                    params = {
                        'language': 'unk',
                        'detectOrientation': 'true'
                    }
                    ocr_url = "https://aivisiontests.cognitiveservices.azure.com/vision/v3.0/ocr"
                    response = requests.post(ocr_url,
                    headers=headers,
                    params=params,
                    data=encoded_image
                    )
                    content = json.loads(response.content)
                    regions = content["regions"]
                    if regions is not None:
                        for region in regions:
                            lines = region["lines"]
                            text = ""
                            for line in lines:
                                words = line["words"]
                                text = ""
                                for word in words:
                                    bbox = [int(num) for num in word["boundingBox"].split(",")]
                                    text = text + word["text"].upper()
                                    print("Bounding box: {0} {1}".format(bbox[2], bbox[3]))
                                    print("Text: {}".format(text))
                                    origin = (bottom, left - 10)
                            if text is not None:
                                font = cv2.FONT_HERSHEY_SIMPLEX
                                fontScale = 2
                                thickness = 2
                                cvImage = cv2.putText(
                                    cvImage,
                                    text,
                                    origin,
                                    font,
                                    fontScale,
                                    (0, 255, 0),
                                    thickness,
                                    cv2.LINE_AA
                                )
                except Exception as ex:
                    print("Error:", str(ex))                
    return cvImage


def car_detection_on_frame(cvImage):
    output_path = "/home/visionadmin/source/cropped_license_plates"
    detectedObjects = inferenceEngine.score(cvImage)
    if len(detectedObjects) > 0:
        confidence = 0.3
        results = []
        height, width, channels = cvImage.shape
        for detected in detectedObjects:
            label = float(detected["entity"]["tag"]["value"])
            if label == 1.0: # 1.0=Car; 2.0=Licence plate
                conf = float(detected["entity"]["tag"]["confidence"])
                if conf > confidence:
                    results.append(detected)
        for result in results:
            bbox = result['entity']['box']
            left = float(bbox['l'])
            top = float(bbox['t'])
            bbox_width = float(bbox['w'])
            bbox_height = float(bbox['h'])
            left = math.ceil(left * width)
            top = math.ceil(top * height)
            bbox_width = math.ceil(float(bbox['w']) * width)
            bbox_height = math.ceil(float(bbox['h']) * height)
            bottom = top + bbox_height
            right = left + bbox_width
            color = (255, 0, 0)  # Blue in BGR
            top_left = (left, top)
            bottom_right = (right, bottom)
            thickness = 2
            cvImage = cv2.rectangle(cvImage, top_left, bottom_right, color, thickness)
    return cvImage


def lpr_recognition_from_frame(frame):
    frame = car_detection_on_frame(frame)
    frame = openvino_cognitiveservices_license_plate_recognition_on_frame(frame)
    return frame


def uncanny_detection_from_frame(cvImage):
    response = None
    file_name = "frame.jpg"
    cv2.imwrite(file_name, cvImage)
    try:
        url = "http://52.188.213.201:5001/api/upload_event"
        files = {
            'file': open(file_name, 'rb')
        }
        response = requests.post(url, files=files)
        print(response.text)
        detectedObjects = json.loads(response.text)
        detectedObjects = detectedObjects['inferences']
        return detectedObjects
    except Exception as ex:
        print("UnCanny error:", str(ex))
    return []


# Initial settings of AI model
#IS_MODEL_NAME = "person-vehicle-bike-detection-crossroad-1016" # see MLModel class for full list of models and other possibilities
IS_MODEL_NAME = "vehicle-license-plate-detection-barrier-0106" # see MLModel class for full list of models and other possibilities
RECOGNITION_MODEL_NAME = "license-plate-recognition-barrier-0001"
IS_TARGET_DEVICE = "CPU"
IS_MODEL_PRECISION = "FP32"
MODEL_PRECISION_16 = "FP16"

app = Flask(__name__)
inferenceEngine = MLModel(    modelName=IS_MODEL_NAME, 
                                modelPrecision=IS_MODEL_PRECISION, 
                                targetDev=IS_TARGET_DEVICE, 
                            )


@app.route('/stream/<id>')
def stream(id):
    respBody = ("<html>"
               "<h1>Stream with inferencing overlays</h1>"
               "img src=\"/mjpeg/" + id + "\"/>"
               "</html>")
    return Response(respBody, status=200)


@app.route("/score", methods = ['POST'])
def scoreRRS():
    global inferenceEngine
 
    try:
        # get request as byte stream
        reqBody = request.get_data(False)
        # convert from byte stream
        inMemFile = io.BytesIO(reqBody)
        # load a sample image
        inMemFile.seek(0)
        fileBytes = np.asarray(bytearray(inMemFile.read()), dtype=np.uint8)
        cvImage = cv2.imdecode(fileBytes, cv2.IMREAD_COLOR)

        if request.args:
            stream = request.args.get('stream')
        try:
            if stream is not None:
                output_img = lpr_recognition_from_frame(cvImage)
                output_img = Image.fromarray(output_img)
                imgBuf = io.BytesIO()
                output_img.save(imgBuf, format='JPEG')
                headers = {'Content-Type': 'image/jpeg'}
                # post the image with bounding boxes so that it can be viewed as an MJPEG stream
                postData = b'--boundary\r\n' + b'Content-Type: image/jpeg\r\n\r\n' + imgBuf.getvalue() + b'\r\n'
                requests.post('http://127.0.0.1:5001/mjpeg_pub/' + stream, data = postData)
        except Exception as ex:
            print('EXCEPTION:', str(ex))
                
        # call scoring function
        detectedObjects = inferenceEngine.score(cvImage)
        if len(detectedObjects) > 0:
            confidence = 0.3
            results = []
            for detected in detectedObjects:
                conf = float(detected["entity"]["tag"]["confidence"])
                if  conf > confidence:
                    results.append(detected)
            respBody = {                    
                        "inferences" : results
                    }

            logging.info("[AI EXT] Sending response.")
            respBody = json.dumps(respBody)
            return Response(respBody, status= 200, mimetype ='application/json')
        else:
            logging.info("[AI EXT] Sending empty response.")
            return Response(status= 204)

    except:
        PrintGetExceptionDetails()
        return Response(response='Exception occured while processing the image.', status=500)
    
@app.route("/")
def healthy():
    return "Healthy"

# About
@app.route('/about', methods = ['GET'])
def about_request():
    global inferenceEngine
    return inferenceEngine.about()

if __name__ == "__main__":      
    app.run(host='127.0.0.1', port=5444)


Overwriting inferenceserver/app.py


In the cell above, 5444 is the internal port of the webserver app that listens the requests. Next, we will map it to different ports to expose it externally.

In [3]:
%%writefile $isSolutionPath/wsgi.py
from app import app as application

def create():
    application.run(host='127.0.0.1', port=5444)

Overwriting inferenceserver/wsgi.py


In [4]:
import os
os.makedirs(os.path.join(isSolutionPath, "nginx"), exist_ok=True)

The exposed port of the web app is now 5001, while the internal one is still 5444.

In [5]:
%%writefile $isSolutionPath/nginx/app
server {
    listen 5001;
    server_name _;
 
    location / {
        include proxy_params;
        proxy_pass http://127.0.0.1:5444;
        proxy_connect_timeout 5000s;
        proxy_read_timeout 5000s;
    }
    
     location ~ /mjpeg_pub/(\w+)$ {
         nchan_publisher;
         nchan_channel_id $1;
         nchan_channel_group mjpeg;
         nchan_message_buffer_length 5;
         nchan_message_timeout 10s;
     }

    location ~ /mjpeg/(\w+)$ {
        add_header Content-Type "multipart/x-mixed-replace; boundary=--boundary";
        nchan_subscriber http-raw-stream;
        nchan_channel_id $1;
        nchan_channel_group mjpeg;
        nchan_subscriber_first_message newest;
    }        
}

Overwriting inferenceserver/nginx/app


In [6]:
%%writefile $isSolutionPath/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Overwriting inferenceserver/gunicorn_logging.conf


In [7]:
%%writefile $isSolutionPath/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('[AI EXT] READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('[AI EXT] Terminating supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('[AI EXT] Could not terminate supervisor: ' + e.strerror + '\n')
            write_stdout('[AI EXT] RESULT 2\nOK')

main()

Overwriting inferenceserver/kill_supervisor.py


In [8]:
import os
os.makedirs(os.path.join(isSolutionPath, "etc"), exist_ok=True)

In [9]:
%%writefile $isSolutionPath/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

environment=LD_LIBRARY_PATH=%(ENV_LD_LIBRARY_PATH)s,INTEL_CVSDK_DIR=%(ENV_INTEL_CVSDK_DIR)s,OpenCV_DIR=%(ENV_OpenCV_DIR)s,InferenceEngine_DIR=%(ENV_InferenceEngine_DIR)s,PYTHONPATH=%(ENV_PYTHONPATH)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s,HDDL_INSTALL_DIR=%(ENV_HDDL_INSTALL_DIR)s,INTEL_OPENVINO_DIR=%(ENV_INTEL_OPENVINO_DIR)s,PATH=%(ENV_PATH)s

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/isserver
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/isserver
events=PROCESS_STATE_FATAL
priority=2

Overwriting inferenceserver/etc/supervisord.conf


In [10]:
%%writefile $isSolutionPath/requirements.txt
pillow<7.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3
urllib3

Overwriting inferenceserver/requirements.txt


## Create a Docker File to Containerize the ML Solution and Web App Server

> <span style="color:red; font-weight: bold; font-size:1.1em;"> [!IMPORTANT] </span>  

> The OpenVINO™ Toolkit is a licenced software. To ensure that you are using the latest version of the OpenVINO™ Toolkit, follow these instructions to obtain a licensed download link:  

> 1) Go to the [Intel donwload link](https://software.intel.com/en-us/openvino-toolkit/choose-download/free-download-linux) for the OpenVINO™ Toolkit

> 2) Click on the "Register & Download" button  

> <img src="../../../../images/_openvino_img_03_001.jpg" width=400 alt="> Figure: Register & Download."/>  

> 3) Fill in the form and click submit 

> <img src="../../../../images/_openvino_img_03_002.jpg" width=400 alt="> Figure: Fill the form."/>  

> 4) Over the "Full Package" link, right click and get the link which should look like something:  
    http://registrationcenter-download.intel.com/akdlm/irc_nas/<SOMECODE\>/l_openvino_toolkit_p_2020.3.194.tgz  
    
> <img src="../../../../images/_openvino_img_03_003.jpg" width=400 alt="> Figure: Download link."/>  

> 5) In the below cell, set the value of variable "openVinoToolkitDownloadLink" to the download link you have.

In [11]:
# As described above, set the value of variable "openVinoToolkitDownloadLink" to the download link you have (below is sample URI, just remove it and use your own address)

# openVinoToolkitDownloadLink = "http://registrationcenter-download.intel.com/akdlm/irc_nas/<SOMECODE>/l_openvino_toolkit_p_2020.3.194.tgz"


In [12]:
%%writefile $isSolutionPath/Dockerfile

FROM ubuntu:18.04

USER root

ARG WORK_DIR=/isserver
ENV WORK_DIR ${WORK_DIR}
ENV PATH /opt/miniconda/bin:${PATH}

RUN mkdir -p ${WORK_DIR}

WORKDIR ${WORK_DIR}

#
# Install base
#
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        wget \
        locales \
        # Python environment
        python3 \
        python3-setuptools &&\
    #
    # Dependencies: conda
    wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh --no-check-certificate &&\ 
    /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/miniconda &&\
    #
    # Cleaning
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Install OpenVINO™
#
#COPY l_openvino_toolkit_p_2020.1.023.tgz ${WORK_DIR}
RUN apt-get update &&\
    apt-get install -y --no-install-recommends \
        # Essentials
        cpio \
        udev \
        unzip \
        autoconf \
        automake \
        libtool

RUN wget --quiet IS_OPENVINO_TOOLKIT_DOWNLOAD_LINK -O ${WORK_DIR}/l_openvino_toolkit_p_2020.1.023.tgz &&\
    pattern="COMPONENTS=DEFAULTS" &&\
    replacement="COMPONENTS=intel-openvino-ie-sdk-ubuntu-bionic__x86_64;intel-openvino-ie-rt-cpu-ubuntu-bionic__x86_64;intel-openvino-ie-rt-vpu-ubuntu-bionic__x86_64;intel-openvino-opencv-lib-ubuntu-bionic__x86_64" &&\
    tar -xzf l_openvino_toolkit*.tgz &&\
    rm -rf l_openvino_toolkit*.tgz &&\
    cd l_openvino_toolkit* &&\
    sed -i "s/$pattern/$replacement/" silent.cfg &&\
    sed -i "s/decline/accept/g" silent.cfg &&\
    /bin/bash ./install.sh -s silent.cfg &&\
    cd - &&\
    cd /opt/intel/openvino/install_dependencies &&\
    /bin/bash ./install_openvino_dependencies.sh &&\
    # setup environment variables
    echo "source /opt/intel/openvino/bin/setupvars.sh" >> /root/.bashrc &&\
    #
    # Cleaning
    cd ${WORK_DIR} &&\
    rm -rf * &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

#
# Set environment variables as in ${INTEL_OPENVINO_DIR}/bin/setupvars.sh
ENV INTEL_OPENVINO_DIR /opt/intel/openvino
ENV LD_LIBRARY_PATH ${INTEL_OPENVINO_DIR}/opencv/lib:${INTEL_OPENVINO_DIR}/deployment_tools/ngraph/lib:/opt/intel/opencl:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/gna/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/mkltiny_lnx/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/tbb/lib:${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/lib/intel64
ENV INTEL_CVSDK_DIR ${INTEL_OPENVINO_DIR}
ENV OpenCV_DIR ${INTEL_OPENVINO_DIR}/opencv/cmake
ENV InferenceEngine_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/share
ENV PYTHONPATH ${INTEL_OPENVINO_DIR}/python/python3.7:${INTEL_OPENVINO_DIR}/python/python3:${INTEL_OPENVINO_DIR}/deployment_tools/open_model_zoo/tools/accuracy_checker:${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer
ENV PATH ${INTEL_OPENVINO_DIR}/deployment_tools/model_optimizer:/opt/miniconda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:${PATH}
ENV HDDL_INSTALL_DIR ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/hddl

#
# Exclude UDEV by rebuilding libusb without UDEV support
RUN cp ${INTEL_OPENVINO_DIR}/deployment_tools/inference_engine/external/97-myriad-usbboot.rules /etc/udev/rules.d/ &&\
    ldconfig &&\
    cd /opt && wget --quiet --no-check-certificate http://github.com/libusb/libusb/archive/v1.0.22.zip -O /opt/v1.0.22.zip &&\
    unzip v1.0.22.zip && cd libusb-1.0.22 &&\
    ./bootstrap.sh &&\
    ./configure --disable-udev --enable-shared &&\
    make -j4

RUN apt-get update &&\
    apt-get install -y --no-install-recommends libusb-1.0-0-dev &&\
    cd /opt &&\
    rm -rf /var/lib/apt/lists/* &&\
    cd /opt/libusb-1.0.22/libusb &&\
    /bin/mkdir -p '/usr/local/lib' &&\
    /bin/bash ../libtool --mode=install /usr/bin/install -c libusb-1.0.la '/usr/local/lib' &&\
    /bin/mkdir -p '/usr/local/include/libusb-1.0' &&\
    /usr/bin/install -c -m 644 libusb.h '/usr/local/include/libusb-1.0' &&\
    /bin/mkdir -p '/usr/local/lib/pkgconfig' &&\
    cd /opt/libusb-1.0.22/ &&\
    /usr/bin/install -c -m 644 libusb-1.0.pc '/usr/local/lib/pkgconfig' &&\
    ldconfig

#
# Install ML solution
RUN apt-get update && apt-get install -y --no-install-recommends \
    nginx \
    supervisor &&\
    pip install \
        numpy \
        azure-iot-device

# Install Nchan module
RUN apt-get update -y &&\
    apt-get install -y libnginx-mod-nchan &&\
    /etc/init.d/nginx restart

ADD . ${WORK_DIR}
ADD etc /etc

RUN rm -rf /var/lib/apt/lists/* &&\
    rm /etc/nginx/sites-enabled/default &&\
    cp ${WORK_DIR}/nginx/app /etc/nginx/sites-available/ &&\
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&\
    pip install -r ${WORK_DIR}/requirements.txt &&\
    /opt/miniconda/bin/conda clean -ya &&\
    rm -rf /opt/miniconda/pkgs &&\
    rm -rf /var/lib/apt/lists/*

EXPOSE 5001 8080
CMD ["supervisord", "-c", "/isserver/etc/supervisord.conf"]


Overwriting inferenceserver/Dockerfile


In [13]:
# Update Docker file with custom environment variable: IoT Edge device's connection string
filePath = isSolutionPath+"/Dockerfile"
file = open(filePath)
dockerFileTemplate = file.read()
dockerFileTemplate = dockerFileTemplate.replace("IS_OPENVINO_TOOLKIT_DOWNLOAD_LINK", "\""+openVinoToolkitDownloadLink+"\"")

with open(filePath, 'wt', encoding='utf-8') as outputFile:
    outputFile.write(dockerFileTemplate)

## Create a Local Docker Image
Finally, we will create a Docker image locally. We will later host the image in a container registry like Docker Hub, Azure Container Registry, or a local registry.

To run the following code snippet, you must have the pre-requisities mentioned in [the requirements page](../common/requirements.md). Most notably, we are running the `docker` command without `sudo`.

> <span>[!WARNING]</span>
> Please ensure that Docker is running before executing the cell below. Execution of the cell below may take several minutes. 

In [14]:
!sudo docker build -t $containerImageName --file ./$isSolutionPath/Dockerfile ./$isSolutionPath

Sending build context to Docker daemon  202.1MB
Step 1/27 : FROM ubuntu:18.04
 ---> d27b9ffc5667
Step 2/27 : USER root
 ---> Using cache
 ---> 30fea6fec12d
Step 3/27 : ARG WORK_DIR=/isserver
 ---> Using cache
 ---> e051f506fe89
Step 4/27 : ENV WORK_DIR ${WORK_DIR}
 ---> Using cache
 ---> 5800225e4748
Step 5/27 : ENV PATH /opt/miniconda/bin:${PATH}
 ---> Using cache
 ---> 5d2b2bed46a5
Step 6/27 : RUN mkdir -p ${WORK_DIR}
 ---> Using cache
 ---> 1d82213511af
Step 7/27 : WORKDIR ${WORK_DIR}
 ---> Using cache
 ---> b83b980f1afa
Step 8/27 : RUN apt-get update &&    apt-get install -y --no-install-recommends         wget         locales         python3         python3-setuptools &&    wget --quiet https://repo.anaconda.com/miniconda/Miniconda3-4.5.11-Linux-x86_64.sh -O ${WORK_DIR}/miniconda.sh --no-check-certificate &&    /bin/bash ${WORK_DIR}/miniconda.sh -b -p /opt/miniconda &&    /opt/miniconda/bin/conda clean -ya &&    rm -rf /opt/miniconda/pkgs &&    rm -rf /var/lib/apt/lists/*
 ---> Us

## Next Steps
If all the code cells above have successfully finished running, return to the Readme page to continue.   